In [1]:
1+1

2

In [14]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMessageTermination,MaxMessageTermination

from dotenv import load_dotenv
load_dotenv()
import os

api_key=os.getenv("GAMINI_API_KEY")

In [3]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key,
)

In [9]:
dsa_solver=AssistantAgent(
    name="dsa_solver",
    model_client=model_client,
    system_message="you give me code in c++ to solve complex dsa problem"
)

code_reviewer=AssistantAgent(
    name="recviewer",
    model_client=model_client,
    system_message="you review the code by the dsa_solver "\
    "is code is crrect please say 'Terminate'"
)
code_editer=AssistantAgent(
    name="code_editer",
    model_client=model_client,
    system_message="you make the code easy to understand and add comment of the code "\
    "is code is crrect please say 'Terminate'"

) 

In [15]:
terminate=TextMessageTermination("Terminate") | MaxMessageTermination(5)
from autogen_agentchat.teams import RoundRobinGroupChat
team=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editer],
    # max_turns=3/
    termination_condition=terminate
)

In [11]:
from autogen_agentchat.messages import TextMessage

async def test_team():
    task=TextMessage(content="write code to of find prime number in c++",source='user')
    result= await team.run(task=task)

    for each_messages in result.messages:
        print(f'{each_messages.source}:{each_messages.content}')

In [16]:
from autogen_agentchat.ui import Console
await team.reset()
await Console(team.run_stream(task="how to find even or odd code"))

---------- TextMessage (user) ----------
how to find even or odd code


---------- TextMessage (dsa_solver) ----------
There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:

**1. Using the modulo operator (%)**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if (number % 2 == 0) {
    cout << number << " is even." << endl;
  } else {
    cout << number << " is odd." << endl;
  }

  return 0;
}
```

* **Explanation:** The modulo operator (%) gives you the remainder of a division.  If a number is perfectly divisible by 2 (remainder is 0), it's even. Otherwise, it's odd.


**2. Using a bitwise AND operator (&)**

```C++
#include <iostream>

using namespace std;

int main() {
  int number;

  cout << "Enter an integer: ";
  cin >> number;

  if ((number & 1) == 0) {
    cout << number << " is even." << endl;
  } else {
    cout << number << " is odd." << endl;
  }

  return 0;
}
```

* **Explanation:**  This method is generall

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 23, 7, 38, 40, 461054, tzinfo=datetime.timezone.utc), content='how to find even or odd code', type='TextMessage'), TextMessage(source='dsa_solver', models_usage=RequestUsage(prompt_tokens=20, completion_tokens=921), metadata={}, created_at=datetime.datetime(2025, 6, 23, 7, 38, 45, 936277, tzinfo=datetime.timezone.utc), content='There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:\n\n**1. Using the modulo operator (%)**\n\n```C++\n#include <iostream>\n\nusing namespace std;\n\nint main() {\n  int number;\n\n  cout << "Enter an integer: ";\n  cin >> number;\n\n  if (number % 2 == 0) {\n    cout << number << " is even." << endl;\n  } else {\n    cout << number << " is odd." << endl;\n  }\n\n  return 0;\n}\n```\n\n* **Explanation:** The modulo operator (%) gives you the remainder of a division.  If a number is perfectly 

## Managing state f team and agent

In [18]:
agent_state= await dsa_solver.save_state()

In [19]:
agent_state

{'type': 'AssistantAgentState',
 'version': '1.0.0',
 'llm_context': {'messages': [{'content': 'how to find even or odd code',
    'source': 'user',
    'type': 'UserMessage'},
   {'content': 'There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:\n\n**1. Using the modulo operator (%)**\n\n```C++\n#include <iostream>\n\nusing namespace std;\n\nint main() {\n  int number;\n\n  cout << "Enter an integer: ";\n  cin >> number;\n\n  if (number % 2 == 0) {\n    cout << number << " is even." << endl;\n  } else {\n    cout << number << " is odd." << endl;\n  }\n\n  return 0;\n}\n```\n\n* **Explanation:** The modulo operator (%) gives you the remainder of a division.  If a number is perfectly divisible by 2 (remainder is 0), it\'s even. Otherwise, it\'s odd.\n\n\n**2. Using a bitwise AND operator (&)**\n\n```C++\n#include <iostream>\n\nusing namespace std;\n\nint main() {\n  int number;\n\n  cout << "Enter an integer: ";\n  cin >> number;\n\n 

In [20]:
new_dsa_solver=AssistantAgent(name="javadsasolver",model_client=model_client)
new_dsa_solver.load_state(agent_state)

<coroutine object AssistantAgent.load_state at 0x000001B7DDFEF030>

In [21]:
team_state=await team.save_state()

In [22]:
team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'dsa_solver': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'how to find even or odd code',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'There are several ways to determine if a number is even or odd in C++.  Here are a few, with explanations:\n\n**1. Using the modulo operator (%)**\n\n```C++\n#include <iostream>\n\nusing namespace std;\n\nint main() {\n  int number;\n\n  cout << "Enter an integer: ";\n  cin >> number;\n\n  if (number % 2 == 0) {\n    cout << number << " is even." << endl;\n  } else {\n    cout << number << " is odd." << endl;\n  }\n\n  return 0;\n}\n```\n\n* **Explanation:** The modulo operator (%) gives you the remainder of a division.  If a number is perfectly divisible by 2 (remainder is 0), it\'s even. Otherwise, it\'s odd.\n\n\n**2. Using a bit